In [1]:
import numpy as np
import seaborn as sns
import matplotlib as mpl, matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

In [2]:
def getData(file_path):
    runs = []
    values = []
    with open(file_path) as f:
        for l in f:
            if l == "\n":
                runs.append(values[:])
                values.clear()
                continue

            error = l.split(" ")
            error = [float(w.replace("\n", "")) for w in error]
            values.append(error[-1])
    return runs

def get_Results(data):
    results = {'Best': [], 'Worst': [], 'Median': [], 'Mean': [], 'Std.': []}
    
    for i in range(len(data)):
        results['Best'].append(np.min(data[i]))
        results['Worst'].append(np.max(data[i]))
        results['Median'].append(np.median(data[i]))
        results['Mean'].append(np.mean(data[i]))
        results['Std.'].append(np.std(data[i]))
        
    return results
    
def get_jSO_Results(file_path):
    results = {'Best': [], 'Worst': [], 'Median': [], 'Mean': [], 'Std.': []}
    
    with open(file_path) as f:
        for l in f:
            if l == "\n":
                return results
            
            words = l.split(" ")
            #words = [float(w.replace("\n", "")) for w in words]
            results['Best'].append(words[1])
            results['Worst'].append(words[2])
            results['Median'].append(words[3])
            results['Mean'].append(words[4])
            results['Std.'].append(words[5])
        return results
    
def print_comparsion(statistic):
    for func in range(30):
        print("f"+ str(func+1),"{:.4e}".format(float(jSO_d10[statistic][func])) , "{:.4e}".format(m_1_e10_d10[statistic][func]), "{:.4e}".format(m_c_e10_d10[statistic][func]), "{:.4e}".format(x_c_e10_d10[statistic][func]), end="")
        print()

In [45]:
jSO = {}
jSO['d10'] = get_jSO_Results('../Codes-of-Top-Methods-and-results/jSO-SOURCE-RESULTS/D10.rez')
jSO['d30'] = get_jSO_Results('../Codes-of-Top-Methods-and-results/jSO-SOURCE-RESULTS/D30.rez')
jSO['d50'] = get_jSO_Results('../Codes-of-Top-Methods-and-results/jSO-SOURCE-RESULTS/D50.rez')
jSO['d100'] = get_jSO_Results('../Codes-of-Top-Methods-and-results/jSO-SOURCE-RESULTS/D100.rez')

In [19]:
jSO['d10']['Mean'] = alg_data['10']['20']['0.5']['J']['Mean']

In [44]:
bkp = jSO['d10']['Mean']

In [6]:
pfx_path = "C:/Users/rapha/Documents/Estudos (Doutorado)/Resultados/Experimento I/E"

In [113]:
def get_alg_data_expI(dim, els, per, alg, alg_data):
    if dim not in alg_data: 
        alg_data[dim] = {}
        
    if els not in alg_data[dim]: 
        alg_data[dim][els] = {}
        
    if per not in alg_data[dim][els]:
        alg_data[dim][els][per] = {}
            
    #if dim not in alg_data[els][per]:
    #    alg_data[dim][els][per][alg] = {}
        
    alg_data[dim][els][per][alg] = get_Results(getData(pfx_path + els + "/" + per + "/" + alg  +  "/rawDataD" + dim + ".dat"))
    
    global contador
    cmb_dict[contador] = (els, per, alg)
    contador = contador + 1
    
    return alg_data

def print_results(statistic, D, E, data_size):
    wins = {}
    vals = {}
    
    wins['jSO'] = 0
    for cmb in range(data_size):
        wins[cmb + 1] = 0
    
    print("\\begin{table}[H]\n"
    "\centering\n"
    "\caption{Algorithms comparison for $D=" + str(D) +"$. }\n" 
    "\label{tab:results"+ str(statistic) +"D=" + str(D) +"}\n"
    "\\begin{tabular}{|l|"+ (data_size+1) * "c|" +"}\n"
    " \hline\n")
    
    print("& jSO ", end="")
    for alg in range(data_size):
        print(" & " + str(alg+1), end="")
    
    print(" \\\\ \n")
    #print("  \hline \hline")
    for func in range(30):
        vals['jSO'] = float(jSO['d' + str(D)][statistic][func])
        for cmb in range(data_size):
            els = cmb_dict[cmb+1][0]
            per = cmb_dict[cmb+1][1]
            alg = cmb_dict[cmb+1][2]
            
            vals[cmb + 1] = alg_data[D][els][per][alg][statistic][func]
            
        vals = {key: value for key, value in sorted(vals.items(), key=lambda item: item[1])}
        sorted_keys = list(vals.keys())  
        
        rank = {}
        pos = 0
        current = -1.0
        count = 0
        _sum = 0
        aux = {}
        for i in range(len(sorted_keys)):
            if vals[sorted_keys[i]] != current:
                pos += 1
                
                count = 1
                _sum = (i+1)
                
                current = vals[sorted_keys[i]]
                rank[sorted_keys[i]] = pos
            else:
                rank[sorted_keys[i]] = pos
                _sum += (i+1)
                count += 1
            
            if count > 0:
                aux[pos] = _sum / count
            else:
                aux[pos] = i+1
                 
        final_rank = {}
        for r in rank:
            final_rank[r] = aux[rank[r]]
        
        key_min = min(vals.keys(), key=(lambda k: vals[k]))
        best = vals[key_min]
            
        wins['jSO'] += final_rank['jSO']
        for cmb in range(data_size):
            wins[cmb + 1] += final_rank[cmb + 1]
                
    print("\hline")
    print("$wins$ ", end="")
    
    key_min = min(wins.keys(), key=(lambda k: wins[k]))
    best = wins[key_min]
        
    if wins['jSO'] == best:
        print(" & \\textbf{" + str(wins['jSO'])+"}", end="")
    else:
        print(" & " + str(wins['jSO']), end="")
            
    for cmb in range(data_size):
        if wins[cmb+1] == best:
            print(" & \\textbf{" + str(wins[cmb+1])+"}", end="")
        else:
            print(" & " + str(wins[cmb+1]), end="")
    
    print(" \\\\")
    print("\hline")
    print("\end{tabular}")
    print("\end{table}")

def print_baseline_comparson(statistic, D, E):
    wins = {}
    vals = {}
     
    #for cv in cv_list:
    #    wins[cv] = 0
        
    for cmb in range(Size):
        wins[cmb + 1] = 0
    
    print("\\begin{table}[H]\n"
    "\centering\n"
    "\caption{Algorithms comparison for $D=" + str(D) +"$. }\n" 
    "\label{tab:results"+ str(statistic) +"D=" + str(D) +"}\n"
    "\\begin{tabular}{|l|"+ (Size+1) * "c|" +"}\n"
    " \hline\n")
    
    for alg in range(Size):
        print(" & " + str(alg+1), end="")
        
    print(" \\\\ \n")
    for func in range(30):
        for cmb in range(Size):
            els = cmb_dict[cmb+1][0]
            per = cmb_dict[cmb+1][1]
            alg = cmb_dict[cmb+1][2]
            
            #print(E, alg, D, percent, strategy, statistic, func)
            
            vals[cmb + 1] = alg_data[D][els][per][alg][statistic][func]
            
            #print(vals[cmb + 1], float(jSO['d'+str(D)][statistic][func]))
            if vals[cmb + 1] <= float(jSO['d'+str(D)][statistic][func]):
                wins[cmb + 1] += 1
        
    print("\hline")
    print("$wins$ ", end="")
    
    key_min = max(wins.keys(), key=(lambda k: wins[k]))
    best = wins[key_min]
        
    for cmb in range(Size):
        if wins[cmb+1] == best:
            print(" & \\textbf{" + str(wins[cmb+1])+"}", end="")
        else:
            print(" & " + str(wins[cmb+1]), end="")
            
    
    print(" \\\\")
    print("\hline")
    print("\end{tabular}")
    print("\end{table}")

In [139]:
els_list = ['20', '40', '80']
alg_list = ['CVTP', 'XCVTP', 'KTP']
dim_list = ['30']
per_list  = ['0.3', '0.5', '0.8']

contador = 1
cmb_dict = {}
alg_data = {}

for dim in dim_list:
    for els in els_list:
        for per in per_list:
            for alg in alg_list:
                print(dim, els, per, alg)
                alg_data = get_alg_data_expI(dim, els, per, alg, alg_data)
                
Size = len(cmb_dict)

30 20 0.3 CVTP
30 20 0.3 XCVTP
30 20 0.3 KTP
30 20 0.5 CVTP
30 20 0.5 XCVTP
30 20 0.5 KTP
30 20 0.8 CVTP
30 20 0.8 XCVTP
30 20 0.8 KTP
30 40 0.3 CVTP
30 40 0.3 XCVTP
30 40 0.3 KTP
30 40 0.5 CVTP
30 40 0.5 XCVTP
30 40 0.5 KTP
30 40 0.8 CVTP
30 40 0.8 XCVTP
30 40 0.8 KTP
30 80 0.3 CVTP
30 80 0.3 XCVTP
30 80 0.3 KTP
30 80 0.5 CVTP
30 80 0.5 XCVTP
30 80 0.5 KTP
30 80 0.8 CVTP
30 80 0.8 XCVTP
30 80 0.8 KTP


In [140]:
CVTP = alg_data['30']['80']['0.8']['CVTP']['Mean']
XCVTP = alg_data['30']['80']['0.8']['XCVTP']['Mean']
KTP = alg_data['30']['40']['0.8']['KTP']['Mean']

In [132]:
for key, value in cmb_dict.items():
    print("\\textbf{" + str(key) + "}: ", end="")
    
    print('(' + value[0]+', ' + value[1] + ', ' + value[2] + ')', end="")
    if key > 0 and key % 3 == 0:
        print(" \\\\")
        print("\hline")
    else:
        print(" &")

\textbf{1}: (20, 0.3, XCVTP) &
\textbf{2}: (20, 0.5, XCVTP) &
\textbf{3}: (20, 0.8, XCVTP) \\
\hline
\textbf{4}: (40, 0.3, XCVTP) &
\textbf{5}: (40, 0.5, XCVTP) &
\textbf{6}: (40, 0.8, XCVTP) \\
\hline
\textbf{7}: (80, 0.3, XCVTP) &
\textbf{8}: (80, 0.5, XCVTP) &
\textbf{9}: (80, 0.8, XCVTP) \\
\hline


In [135]:
print_results('Mean', '30', '20', Size)

\begin{table}[H]
\centering
\caption{Algorithms comparison for $D=30$. }
\label{tab:resultsMeanD=30}
\begin{tabular}{|l|c|c|c|c|c|c|c|c|c|c|}
 \hline

& jSO  & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\ 

\hline
$wins$  & \textbf{102.5} & 228.5 & 161.0 & 108.5 & 230.5 & 171.5 & 124.5 & 229.0 & 172.5 & 121.5 \\
\hline
\end{tabular}
\end{table}


cmb_dict

In [137]:
print_baseline_comparson('Mean', '30', '20')

\begin{table}[H]
\centering
\caption{Algorithms comparison for $D=30$. }
\label{tab:resultsMeanD=30}
\begin{tabular}{|l|c|c|c|c|c|c|c|c|c|c|}
 \hline

 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\ 

\hline
$wins$  & 9 & 11 & \textbf{14} & 9 & 10 & 12 & 8 & 10 & 12 \\
\hline
\end{tabular}
\end{table}


In [34]:
cmb_dict[2]

('40', '0.3', 'XCVTP')

In [145]:
cmb_dict[11]

('80', '0.5', 'XCVTP')

In [147]:
cmb_dict[6]

('40', '0.3', 'KTP')

In [143]:
def print_results_(statistic, D, E, data_size):
    wins = {}
    vals = {}
    
    wins['jSO'] = 0
    for cmb in ['CVTP', 'XCVTP', 'KTP']:
        wins[cmb] = 0
    
    print("\\begin{table}[H]\n"
    "\centering\n"
    "\caption{Algorithms comparison for $D=" + str(D) +"$. }\n" 
    "\label{tab:results"+ str(statistic) +"D=" + str(D) +"}\n"
    "\\begin{tabular}{|l|"+ 4 * "c|" +"}\n"
    " \hline\n")
    
    print("& jSO ", end="")
    for alg in ['CVTP', 'XCVTP', 'KTP']:
        print(" & " + alg, end="")
    
    print(" \\\\ \n")
    #print("  \hline \hline")
    for func in range(30):
        vals['jSO'] = float(jSO['d' + str(D)][statistic][func])
        vals['CVTP'] = CVTP[func]
        vals['XCVTP'] = XCVTP[func]
        vals['KTP'] = KTP[func]
            
        vals = {key: value for key, value in sorted(vals.items(), key=lambda item: item[1])}
        sorted_keys = list(vals.keys())  
        
        rank = {}
        pos = 0
        current = -1.0
        count = 0
        _sum = 0
        aux = {}
        for i in range(len(sorted_keys)):
            if vals[sorted_keys[i]] != current:
                pos += 1
                
                count = 1
                _sum = (i+1)
                
                current = vals[sorted_keys[i]]
                rank[sorted_keys[i]] = pos
            else:
                rank[sorted_keys[i]] = pos
                _sum += (i+1)
                count += 1
            
            if count > 0:
                aux[pos] = _sum / count
            else:
                aux[pos] = i+1
                 
        final_rank = {}
        for r in rank:
            final_rank[r] = aux[rank[r]]
        
        key_min = min(vals.keys(), key=(lambda k: vals[k]))
        best = vals[key_min]
            
        wins['jSO'] += final_rank['jSO']
        wins['CVTP']  += final_rank['CVTP']
        wins['XCVTP']  += final_rank['XCVTP']
        wins['KTP'] += final_rank['KTP']
                
    print("\hline")
    print("$wins$ ", end="")
    
    key_min = min(wins.keys(), key=(lambda k: wins[k]))
    best = wins[key_min]
        
    if wins['jSO'] == best:
        print(" & \\textbf{" + str(wins['jSO'])+"}", end="")
    else:
        print(" & " + str(wins['jSO']), end="")
        
    if wins['CVTP'] == best:
        print(" & \\textbf{" + str(wins['CVTP'])+"}", end="")
    else:
        print(" & " + str(wins['CVTP']), end="")
        
    if wins['XCVTP'] == best:
        print(" & \\textbf{" + str(wins['XCVTP'])+"}", end="")
    else:
        print(" & " + str(wins['XCVTP']), end="")
        
    if wins['KTP'] == best:
        print(" & \\textbf{" + str(wins['KTP'])+"}", end="")
    else:
        print(" & " + str(wins['KTP']), end="")
    
    
    print(" \\\\")
    print("\hline")
    print("\end{tabular}")
    print("\end{table}")

In [110]:
print_results_('Mean', '10', '20', Size)

\begin{table}[H]
\centering
\caption{Algorithms comparison for $D=10$. }
\label{tab:resultsMeanD=10}
\begin{tabular}{|l|c|c|c|c|}
 \hline

& jSO  & CVTP & XCVTP & KTP \\ 

\hline
$wins$  & 74.0 & \textbf{72.0} & 82.0 & \textbf{72.0} \\
\hline
\end{tabular}
\end{table}


In [144]:
print_results_('Mean', '30', '20', Size)

\begin{table}[H]
\centering
\caption{Algorithms comparison for $D=30$. }
\label{tab:resultsMeanD=30}
\begin{tabular}{|l|c|c|c|c|}
 \hline

& jSO  & CVTP & XCVTP & KTP \\ 

\hline
$wins$  & \textbf{62.5} & 65.5 & 86.5 & 85.5 \\
\hline
\end{tabular}
\end{table}
